In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
all_contracts = all_instruments("Future")
match = r'\w{1,2}88'
continious_contracts = all_contracts[all_contracts.order_book_id.str.contains(match)]
continious_contracts = continious_contracts.where(continious_contracts["product"] == "Commodity").dropna(how="all")
all_commodity = sorted(set(continious_contracts.underlying_symbol))

In [14]:
continious_contracts.set_index("order_book_id",inplace = True)

In [ ]:
continious_contracts.symbol.apply(lambda x : x if "螺纹钢" in x else np.nan).dropna()

In [52]:
# 螺纹钢，铁矿石，焦炭
pairs1 = ["RB","I","J"]
# 焦煤，焦炭
pairs2 = ["JM","J"]
# 鸡蛋，玉米，豆粕
pairs3 = ["JD","M","C"]
# 强麦，硬麦
pairs4 = ["WH","WT"]
# 银，金
pairs5 = ["AG","AU"]
# 铜，锌
pairs6 = ["CU","ZN"]

In [53]:
pairs = pairs5

In [74]:
def get_efficient(pairs):
  close_price = list(map(lambda x: get_price(x+"88","2015-01-01","2017-01-01",fields="close"),pairs))
  pairs_data = pd.concat([close_price[0],close_price[1]],axis=1).dropna()
  pairs_data.columns = [pairs[0],pairs[1]]
  params_coef = sm.OLS(pairs_data.iloc[:,1].values,sm.add_constant(pairs_data.iloc[:,0].values)).fit().params
  
  if params_coef[1]>1:
    return {"x": pairs[0],"y":pairs[1],"efficient":int(params_coef[1])}
  params_coef = sm.OLS(pairs_data.iloc[:,0].values,sm.add_constant(pairs_data.iloc[:,1].values)).fit().params
  if params_coef[1]>1:
    return {"x": pairs[1],"y":pairs[0],"efficient":int(params_coef[1])}
  
  return {"x": pairs[0],"y":pairs[1],"efficient":1}


  

In [77]:
pairs_results = get_efficient(pairs)

In [80]:
pairs_results

{'efficient': 15, 'x': 'AU', 'y': 'AG'}

In [85]:
entry_ratio = 2

In [83]:
y = get_price(pairs_results.get("y")+"88","2015-01-01","2017-01-01",fields="close")
x = get_price(pairs_results.get("x")+"88","2015-01-01","2017-01-01",fields="close")
spread = y - x*pairs_results.get("efficient")

In [96]:
spread_avg = spread.rolling(center=False,window=60).mean()
spread_std = spread.rolling(center=False,window=60).std()
up_point = spread_avg+entry_ratio*spread_std
down_point = spread_avg-entry_ratio*spread_std

In [109]:

signal_B = (spread > up_point).astype(int)
signal = (spread < down_point).astype(int).replace([0,1],[np.nan,-1]).fillna(signal_B)

In [108]:
y_chg = y.pct_change()
x_chg = x.pct_change()

In [114]:
combined_data = pd.concat([signal,y_chg,x_chg],axis=1)